<a href="https://colab.research.google.com/github/chrisseub/ML-Practice-Rep/blob/main/SPOTIFY_MUSIC_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Libraries

In [ ]:
!pip install spotipy
import spotipy
import socket
import webbrowser
import threading

def find_available_port():
    """Finds an available port on the localhost."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.bind(('localhost', 0))  # Bind to port 0 to get an available port
        _, port = sock.getsockname()
    return port
# Find an available port
port = find_available_port()
# Update redirect URL with the available port
def start_auth_flow(port):
    redirect_uri = f"http://localhost:{port}/callback"  # Changed redirect_url to redirect_uri
    from spotipy.oauth2 import SpotifyOAuth # Connect to Spotify API

    # Initialize SpotifyOAuth within the function
    oauth = SpotifyOAuth(client_id="7a61ca63ddc54675830839b915bf3f87",
                        client_secret="e724f03e6aa74056bcc6df38ca6b43a7",
                        redirect_uri=f"http://localhost:{port}/callback",
                        scope="user-top-read")

    auth_url = oauth.get_authorize_url() # This line and the following lines were incorrectly indented
    webbrowser.open(auth_url)

    # Create the Spotify client after authentication (code exchange happens in get_access_token)
    global sp  # Make sp accessible from outside the function
    sp = spotipy.Spotify(auth_manager=oauth)

    # You might want to add a signal here to indicate when authentication is complete.
    print("Authentication process complete.")

# Start the authentication flow in a separate thread
auth_thread = threading.Thread(target=start_auth_flow, args=(port,))
auth_thread.start()
auth_thread.join()


#Fetch User's Listening History
# Get user's top tracks
top_tracks = sp.current_user_top_tracks(limit=10)
print("Your favorite songs:")
for i, track in enumerate(top_tracks['items']):
  print(f"{i + 1}. {track['name']} by {track['artists'][0]['name']}")

Authentication process complete.


#Fetch Song Features

In [ ]:
# Get song features for top tracks
print("\nSong Features:")
for track in top_tracks['items']:
  features = sp.audio_features(track['id'])[0]
  print(f"Track: {track['name']} - Danceability: {features['danceability']}, Energy: {features['energy']}")

# Find Similar Songs

In [ ]:
# Recommend songs based on a favorite track favorite_track_id = top_tracks['items'][0]['id'] # Take the first favorite song
favorite_features = sp.audio_features(favorite_track_id)[0] # Search for similar tracks print("\nRecommended Songs:")
results = sp.recommendations(seed_tracks=[favorite_track_id], limit=5)
for track in results['tracks']:
     print(f"{track['name']} by {track['artists'][0]['name']}")
